<a href="https://colab.research.google.com/github/greyhound101/IEEE-CIS-Fraud/blob/master/autoenc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tensorflow==2.1.0-rc0

     |████████████████████████████████| 402.3MB 41kB/s 
     |████████████████████████████████| 3.8MB 52.0MB/s 
     |████████████████████████████████| 450kB 39.9MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=93ed83fee4d067a6b44b7ab17af0c3d1ecf354f7cd6b9780e6174e672f3749ee
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
  Found existing installation: tensorboard 2.2.0
    Uninstalling tensorboard-2.2.0:
      Successfully uninstalled tensorboard-2.2.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0
  Found existing installation: tensorflow 2.2.0rc3
    Uninstalling tensorflow-2.2.0rc3:
      Successfully uninstalled tensorflow-2.2.0rc

In [2]:
pip install keras==2.3.1


  0% 0.00/3.26M [00:00<?, ?B/s]
100% 3.26M/3.26M [00:00<00:00, 108MB/s]
  0% 0.00/1.14M [00:00<?, ?B/s]
100% 1.14M/1.14M [00:00<00:00, 163MB/s]
  0% 0.00/3.21M [00:00<?, ?B/s]
100% 3.21M/3.21M [00:00<00:00, 106MB/s]
 96% 56.0M/58.3M [00:00<00:00, 51.1MB/s]
100% 58.3M/58.3M [00:00<00:00, 85.5MB/s]
 86% 45.0M/52.2M [00:00<00:00, 50.9MB/s]
100% 52.2M/52.2M [00:00<00:00, 88.9MB/s]


In [0]:
import pandas as pd
import numpy as np
trn=pd.read_csv('train_transaction.csv.zip')
tst=pd.read_csv('test_transaction.csv.zip')

In [5]:
ls=list(trn.filter(regex='V'))
trn=trn.drop(ls,1)
tst=tst.drop(ls,1)
trn.shape

(590540, 55)

In [0]:
def ext(fnl):

  fnl['P_1']=fnl['P_emaildomain'].fillna('nan').str.split('.').str[0]
  fnl['P_2']=fnl['P_emaildomain'].fillna('nan').str.split('.').str[1]


  fnl['R_1']=fnl['R_emaildomain'].fillna('nan').str.split('.').str[0]
  fnl['R_2']=fnl['R_emaildomain'].fillna('nan').str.split('.').str[1]
  return fnl
trn=ext(trn)
tst=ext(tst)

In [0]:
trn=trn.drop(['P_emaildomain','R_emaildomain','TransactionDT','TransactionID'],1)
tst=tst.drop(['P_emaildomain','R_emaildomain','TransactionDT','TransactionID'],1)

In [0]:
# def handle_disc(df,df1,cols):
#   for col in cols:
#     df[col]=df[col].fillna('other')
#     cbe=CatBoostEncoder(sigma=0.01)
#     df[col]=cbe.fit_transform(df[col].astype(str),df['isFraud'])
#     df1[col]=cbe.transform(df1[col])
#   return df,df1
def handle_disc(df,cols,num):
  for col in cols:
    par=min(df[col].nunique(),num)
    unq = df[col].value_counts().head(par)
    dum=pd.get_dummies(df[col])[unq.index]
    ls=[]
    for i in dum.columns:
      ls.append(str(col)+str(i))
    dum.columns=ls
    df=pd.concat([df,dum],1)

  df=df.drop(cols,1)
  sp=df.shape[1]
  df.columns=list(range(sp))
  return df
def handle_cont(df,cols):
  for col in cols:
    df[col]=df[col].fillna(df[col].mean())
    mn=df[col].mean()
    std=df[col].std()
    df[col]=(df[col]-mn)/(std+0.1)
    df[col]=np.log10(df[col]+1-min(0,min(df[col])))
  return df[cols]
disc=['ProductCD','card1','card2','card3','card4','card5','card6','addr1','addr2','P_1','P_2','R_1','R_2','M1','M2','M3','M4','M5','M6','M7','M8','M9']
cont=[i for i in tst.columns if i not in disc]
trn_cont=handle_cont(trn,cont)
tst_cont=handle_cont(tst,cont)
trn_cont=trn_cont.replace([np.inf,-np.inf],0)
tst_cont=tst_cont.replace([np.inf,-np.inf],0)
trn_disc=handle_disc(trn,disc,140)
tst_disc=handle_disc(tst,disc,140)

In [0]:
trn=pd.concat([trn_cont,trn_disc],1).reset_index(drop=True)
tst=pd.concat([tst_cont,tst_disc],1).reset_index(drop=True)

In [0]:
ls=list(set(list(tst)).intersection(set(list(trn))))
trn=trn[ls]
tst=tst[ls]


In [0]:
trn=trn.loc[:,~(trn.columns.duplicated())]
tst=tst.loc[:,~(tst.columns.duplicated())]

In [14]:

fnl=pd.concat([trn,tst],0)
fnl=fnl.reset_index(drop=True)
z=fnl.isna().sum()
z[z>0]

Series([], dtype: int64)

In [15]:

from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
Mounted at /content/gdrive


In [16]:

#input fed to model is original data where 10% columns have their values shuffled within themselves 

import gc
import os
gc.collect()
import numpy as np
import keras 
from math import ceil
from keras.models import Model
import matplotlib.pyplot as plt
from keras.utils import Sequence
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Concatenate, Dropout,BatchNormalization
from keras.layers import Add
import keras.backend as K
class DAESequence(Sequence):
    def __init__(self,df,batch_size):
        self.batch_size=batch_size
        self.len_data=df.shape[0]
        self.columns=df.shape[1]
        self.data=df.values
        self.idx=[]
    def __len__(self):
        return int(ceil(self.len_data/self.batch_size))
    def __getitem__(self,idx):
        self.idx.append(idx)
        last=min((idx+1)*self.batch_size,self.len_data)
        idx=idx*self.batch_size
        size=last-idx
        output_x=self.data[idx:last]
        length=last-idx
        rnd=np.random.randint(0,self.len_data-size,size)
        rnd[rnd>idx]+=size
        cols=np.random.randint(0,self.columns,int(self.columns*0.1))
        noise_x=output_x.copy()
        noise_x[:,cols]=self.data[rnd[:,None],cols]
        return noise_x,output_x
from keras import backend as K


class WarmUpLearningRateScheduler(keras.callbacks.Callback):
    """Warmup learning rate scheduler
    """

    def __init__(self, warmup_batches, init_lr, verbose=0):
        """Constructor for warmup learning rate scheduler

        Arguments:
            warmup_batches {int} -- Number of batch for warmup.
            init_lr {float} -- Learning rate after warmup.

        Keyword Arguments:
            verbose {int} -- 0: quiet, 1: update messages. (default: {0})
        """

        super(WarmUpLearningRateScheduler, self).__init__()
        self.warmup_batches = warmup_batches
        self.init_lr = init_lr
        self.verbose = verbose
        self.batch_count = 0
        self.learning_rates = []

    def on_batch_end(self, batch, logs=None):
        self.batch_count = self.batch_count + 1
        lr = K.get_value(self.model.optimizer.lr)
        self.learning_rates.append(lr)

    def on_batch_begin(self, batch, logs=None):
        if self.batch_count <= self.warmup_batches:
            lr = self.batch_count*self.init_lr/self.warmup_batches
            K.set_value(self.model.optimizer.lr, lr)
            if self.verbose > 0:
                print('\nBatch %05d: WarmUpLearningRateScheduler setting learning '
                      'rate to %s.' % (self.batch_count + 1, lr))
warm_up_lr = WarmUpLearningRateScheduler(400, init_lr=0.005)
from keras.models import Sequential
from keras.callbacks.callbacks import ReduceLROnPlateau,ModelCheckpoint
from keras.layers import BatchNormalization
#fnl=pd.concat([fnl_trn,fnl_tst],0).select_dtypes(exclude='uint8')
batch_size=1024
checkpoint_path = "training_1.hdf5" 
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback =ModelCheckpoint(checkpoint_path, 
monitor='loss',save_best_only=True,save_weights_only=True,verbose=1)
len_input_columns=fnl.shape[1]
model_dae = Sequential()
model_dae.add(Dense(units=len_input_columns,input_shape=(len_input_columns,), activation='relu', dtype='float32', name='Input'))
model_dae.add(Dense(units=int(512), activation='relu', dtype='float32',name='Hidden_1'))
model_dae.add(BatchNormalization())
model_dae.add(Dense(units=int(256), activation='relu', dtype='float32',name='Hidden_2'))
model_dae.add(BatchNormalization())
model_dae.add(Dense(units=int(512), activation='relu', dtype='float32',name='Hidden_3'))
model_dae.add(BatchNormalization())
model_dae.add(Dense(units=len_input_columns, activation='linear', dtype='float32', name='Output'))
model_dae.compile(loss='mean_squared_error',optimizer='adam')
hist=model_dae.fit_generator(DAESequence(fnl, batch_size=2048),steps_per_epoch=int(ceil(fnl.shape[0]/batch_size)),epochs=10,workers=2,callbacks=[warm_up_lr])


Using TensorFlow backend.


Epoch 1/10
1072/1072 [==============================] - 36s 33ms/step - loss: 0.0311
Epoch 2/10
1072/1072 [==============================] - 34s 32ms/step - loss: 0.0077
Epoch 3/10
1072/1072 [==============================] - 34s 31ms/step - loss: 0.0064
Epoch 4/10
1072/1072 [==============================] - 34s 31ms/step - loss: 0.0057
Epoch 5/10
1072/1072 [==============================] - 34s 31ms/step - loss: 0.0054
Epoch 6/10
1072/1072 [==============================] - 33s 31ms/step - loss: 0.0052
Epoch 7/10
1072/1072 [==============================] - 33s 31ms/step - loss: 0.0051
Epoch 8/10
1072/1072 [==============================] - 33s 31ms/step - loss: 0.0051
Epoch 9/10
1072/1072 [==============================] - 33s 31ms/step - loss: 0.0049
Epoch 10/10
1072/1072 [==============================] - 33s 31ms/step - loss: 0.0049


In [0]:
path = F"/content/gdrive/My Drive/autoenc_final.hdf5" 
model_dae.save_weights(path)